In [ ]:
import 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import 
import 

from nltk.util import 
from sklearn.feature_extraction.text import 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

### Using NLTK

In [ ]:
# Function to generate n-grams from sentences.
def extract_ngrams(data, num):
    n_grams = ((data), num)
    return [ ' '.join(grams) for grams in n_grams]
 
data = 'A class is a blueprint for the object.'
 
print("1-gram: ", extract_ngrams(data, 1))
print("2-gram: ", extract_ngrams(data, 2))
print("3-gram: ", extract_ngrams(data, 3))
print("4-gram: ", extract_ngrams(data, 4))

In [ ]:
# load dataset
df = pd.read_csv('twitter_training.csv', header=None, usecols=[2,3])

In [ ]:
# preview of dataset
df.head()

In [ ]:
# rename columns
df.columns = ['Sentiment', 'Text']

# remove missing values
df = df.dropna()

# encode target label
le = LabelEncoder()
df['Sentiment'] = le.fit_transform(df['Sentiment'])

# establish input and output
X = list(df['Text'])
y = list(df['Sentiment'])

# split data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Unigram

In [ ]:
# create a bag of words for with unigrams and bigrams
cv = (analyzer = 'word',ngram_range=(1,1), stop_words='english')

# convert training data to bag of words
X_train_cv = cv.(X_train)
X_test_cv = cv.(X_test)

# train naive bayes classifier
clf = ()
clf.(X_train_cv, y_train)

# create predictions
y_pred = clf.(X_test_cv)

# find f-1 score
score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

### Bigram

In [ ]:
# create a bag of words for with unigrams and bigrams
cv = CountVectorizer(analyzer = 'word',ngram_range=(1,2), stop_words='english')

# convert training data to bag of words
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

# train naive bayes classifier
clf = MultinomialNB()
clf.fit(X_train_cv, y_train)

# create predictions
y_pred = clf.predict(X_test_cv)

# find f-1 score
score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

In [ ]:
for N in range(1,11):
    
    # convert training data to bag of words
    cv = CountVectorizer(analyzer = 'word',ngram_range=(1,N), stop_words='english')
    X_train_cv = cv.fit_transform(X_train)
    X_test_cv = cv.transform(X_test)
    
    # train model and generate predictions
    clf = MultinomialNB()
    clf.fit(X_train_cv, y_train)
    y_pred = clf.predict(X_test_cv)
    
    # compute f-1 score
    score = np.round(f1_score(y_test, y_pred, average='micro'), 4)
    print('F-1 score of model with n-gram range of {}: {}'.format((1,N), score))